In [1]:
# Mount this colab file correctly in google drive so I can access relative/local files
from google.colab import drive
drive.mount('/content/gdrive')
% cd /content/gdrive/My Drive/Github/"CS 499 - SPRING 2022"/"1. STAN - Baseline Experiment"
! pwd

# Install python packages I will need in the local environment
! pip install epiweeks
! pip install haversine
! pip install dgl-cu111 -f https://data.dgl.ai/wheels/repo.html

Mounted at /content/gdrive
/content/gdrive/My Drive/Github/CS 499 - SPRING 2022/1. STAN - Baseline Experiment
/content/gdrive/My Drive/Github/CS 499 - SPRING 2022/1. STAN - Baseline Experiment
Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 252.7 MB 49 kB/s 


In [2]:
import time
time_start = time.time()

In [3]:
import csv
import os

import numpy as np
import pandas as pd
from epiweeks import Week

from data_downloader import GenerateTrainingData
from utils import date_today, gravity_law_commute_dist

os.environ['NUMEXPR_MAX_THREADS'] = '16'
os.environ['NUMEXPR_NUM_THREADS'] = '8'

import pickle
import matplotlib.pyplot as plt
import dgl
import torch
from torch import nn
import torch.nn.functional as F
from model import STAN

import sklearn
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [4]:
# GenerateTrainingData().download_jhu_data('2020-05-01', '2020-12-01')
GenerateTrainingData().download_jhu_data('2020-05-01', '2022-03-07')

Finish download


,state,latitude,longitude,fips,date_today,confirmed,deaths,recovered,active,hospitalization,new_cases
0,Alabama,32.3182,-86.9023,1,2020-05-04,8203,8,0.0,7905.0,0.0,226
0,Alabama,32.3182,-86.9023,1,2020-05-05,8520,17,0.0,0.0,0.0,317
0,Alabama,32.3182,-86.9023,1,2020-05-06,8769,28,0.0,0.0,0.0,249
0,Alabama,32.3182,-86.9023,1,2020-05-07,9115,26,0.0,0.0,0.0,346
0,Alabama,32.3182,-86.9023,1,2020-05-08,9437,14,0.0,0.0,0.0,322
...,...,...,...,...,...,...,...,...,...,...,...
57,Wyoming,42.7560,-107.3025,56,2022-03-03,155349,0,0.0,0.0,4.0,72
57,Wyoming,42.7560,-107.3025,56,2022-03-04,155426,0,0.0,0.0,4.0,77
57,Wyoming,42.7560,-107.3025,56,2022-03-05,155426,0,0.0,0.0,2.0,0
57,Wyoming,42.7560,-107.3025,56,2022-03-06,155426,0,0.0,0.0,0.0,0


In [5]:
#Merge population data with downloaded data
raw_data = pickle.load(open('./data/state_covid_data.pickle','rb'))
pop_data = pd.read_csv('./uszips.csv')
pop_data = pop_data.groupby('state_name').agg({'population':'sum', 'density':'mean', 'lat':'mean', 'lng':'mean'}).reset_index()
raw_data = pd.merge(raw_data, pop_data, how='inner', left_on='state', right_on='state_name')

In [6]:
# Generate location similarity
loc_list = list(raw_data['state'].unique())
loc_dist_map = {}

for each_loc in loc_list:
    loc_dist_map[each_loc] = {}
    for each_loc2 in loc_list:
        lat1 = raw_data[raw_data['state']==each_loc]['latitude'].unique()[0]
        lng1 = raw_data[raw_data['state']==each_loc]['longitude'].unique()[0]
        pop1 = raw_data[raw_data['state']==each_loc]['population'].unique()[0]
        
        lat2 = raw_data[raw_data['state']==each_loc2]['latitude'].unique()[0]
        lng2 = raw_data[raw_data['state']==each_loc2]['longitude'].unique()[0]
        pop2 = raw_data[raw_data['state']==each_loc2]['population'].unique()[0]
        
        loc_dist_map[each_loc][each_loc2] = gravity_law_commute_dist(lat1, lng1, pop1, lat2, lng2, pop2, r=0.5)

In [7]:
#Generate Graph
dist_threshold = 18

for each_loc in loc_dist_map:
    loc_dist_map[each_loc] = {k: v for k, v in sorted(loc_dist_map[each_loc].items(), key=lambda item: item[1], reverse=True)}
    
adj_map = {}
for each_loc in loc_dist_map:
    adj_map[each_loc] = []
    for i, each_loc2 in enumerate(loc_dist_map[each_loc]):
        if loc_dist_map[each_loc][each_loc2] > dist_threshold:
            if i <= 3:
                adj_map[each_loc].append(each_loc2)
            else:
                break
        else:
            if i <= 1:
                adj_map[each_loc].append(each_loc2)
            else:
                break

rows = []
cols = []
for each_loc in adj_map:
    for each_loc2 in adj_map[each_loc]:
        rows.append(loc_list.index(each_loc))
        cols.append(loc_list.index(each_loc2))

In [8]:
g = dgl.graph((rows, cols))

In [9]:
#Preprocess features

active_cases = []
confirmed_cases = []
new_cases = []
death_cases = []
static_feat = []

for i, each_loc in enumerate(loc_list):
    active_cases.append(raw_data[raw_data['state'] == each_loc]['active'])
    confirmed_cases.append(raw_data[raw_data['state'] == each_loc]['confirmed'])
    new_cases.append(raw_data[raw_data['state'] == each_loc]['new_cases'])
    death_cases.append(raw_data[raw_data['state'] == each_loc]['deaths'])
    static_feat.append(np.array(raw_data[raw_data['state'] == each_loc][['population','density','lng','lat']]))
    
active_cases = np.array(active_cases)
confirmed_cases = np.array(confirmed_cases)
death_cases = np.array(death_cases)
new_cases = np.array(new_cases)
static_feat = np.array(static_feat)[:, 0, :]
recovered_cases = confirmed_cases - active_cases - death_cases
susceptible_cases = np.expand_dims(static_feat[:, 0], -1) - active_cases - recovered_cases

# Batch_feat: new_cases(dI), dR, dS
#dI = np.array(new_cases)
dI = np.concatenate((np.zeros((active_cases.shape[0],1), dtype=np.float32), np.diff(active_cases)), axis=-1)
dR = np.concatenate((np.zeros((recovered_cases.shape[0],1), dtype=np.float32), np.diff(recovered_cases)), axis=-1)
dS = np.concatenate((np.zeros((susceptible_cases.shape[0],1), dtype=np.float32), np.diff(susceptible_cases)), axis=-1)

In [10]:
#Build normalizer
normalizer = {'S':{}, 'I':{}, 'R':{}, 'dS':{}, 'dI':{}, 'dR':{}}

for i, each_loc in enumerate(loc_list):
    normalizer['S'][each_loc] = (np.mean(susceptible_cases[i]), np.std(susceptible_cases[i]))
    normalizer['I'][each_loc] = (np.mean(active_cases[i]), np.std(active_cases[i]))
    normalizer['R'][each_loc] = (np.mean(recovered_cases[i]), np.std(recovered_cases[i]))
    normalizer['dI'][each_loc] = (np.mean(dI[i]), np.std(dI[i]))
    normalizer['dR'][each_loc] = (np.mean(dR[i]), np.std(dR[i]))
    normalizer['dS'][each_loc] = (np.mean(dS[i]), np.std(dS[i]))

In [11]:
def prepare_data(data, sum_I, sum_R, history_window=5, pred_window=15, slide_step=5):
    # Data shape n_loc, timestep, n_feat
    # Reshape to n_loc, t, history_window*n_feat
    n_loc = data.shape[0]
    timestep = data.shape[1]
    n_feat = data.shape[2]
    
    x = []
    y_I = []
    y_R = []
    y_active_cases = []
    last_I = []
    last_R = []
    concat_I = []
    concat_R = []
    for i in range(0, timestep, slide_step):
        if i+history_window+pred_window-1 >= timestep or i+history_window >= timestep:
            break
        x.append(data[:, i:i+history_window, :].reshape((n_loc, history_window*n_feat)))
        
        concat_I.append(data[:, i+history_window-1, 0])
        concat_R.append(data[:, i+history_window-1, 1])
        last_I.append(sum_I[:, i+history_window-1])
        last_R.append(sum_R[:, i+history_window-1])

        y_I.append(data[:, i+history_window:i+history_window+pred_window, 0])
        y_R.append(data[:, i+history_window:i+history_window+pred_window, 1])
        y_active_cases.append(sum_I[:, i+history_window:i+history_window+pred_window].reshape((n_loc, pred_window)))

    x = np.array(x, dtype=np.float32).transpose((1, 0, 2))
    last_I = np.array(last_I, dtype=np.float32).transpose((1, 0))
    last_R = np.array(last_R, dtype=np.float32).transpose((1, 0))
    concat_I = np.array(concat_I, dtype=np.float32).transpose((1, 0))
    concat_R = np.array(concat_R, dtype=np.float32).transpose((1, 0))
    y_I = np.array(y_I, dtype=np.float32).transpose((1, 0, 2))
    y_R = np.array(y_R, dtype=np.float32).transpose((1, 0, 2))
    y_active_cases = np.array(y_active_cases, dtype=np.float32).transpose((1, 0, 2))

    return x, last_I, last_R, concat_I, concat_R, y_I, y_R, y_active_cases

In [12]:
valid_window = 25
test_window = 25

history_window=6
pred_window=15
slide_step=5

dynamic_feat = np.concatenate((np.expand_dims(dI, axis=-1), np.expand_dims(dR, axis=-1), np.expand_dims(dS, axis=-1)), axis=-1)
    
#Normalize
for i, each_loc in enumerate(loc_list):
    dynamic_feat[i, :, 0] = (dynamic_feat[i, :, 0] - normalizer['dI'][each_loc][0]) / normalizer['dI'][each_loc][1]
    dynamic_feat[i, :, 1] = (dynamic_feat[i, :, 1] - normalizer['dR'][each_loc][0]) / normalizer['dR'][each_loc][1]
    dynamic_feat[i, :, 2] = (dynamic_feat[i, :, 2] - normalizer['dS'][each_loc][0]) / normalizer['dS'][each_loc][1]

dI_mean = []
dI_std = []
dR_mean = []
dR_std = []

for i, each_loc in enumerate(loc_list):
    dI_mean.append(normalizer['dI'][each_loc][0])
    dR_mean.append(normalizer['dR'][each_loc][0])
    dI_std.append(normalizer['dI'][each_loc][1])
    dR_std.append(normalizer['dR'][each_loc][1])

dI_mean = np.array(dI_mean)
dI_std = np.array(dI_std)
dR_mean = np.array(dR_mean)
dR_std = np.array(dR_std)

#Split train-test
train_feat = dynamic_feat[:, :-valid_window-test_window, :]
val_feat = dynamic_feat[:, -valid_window-test_window:-test_window, :]
test_feat = dynamic_feat[:, -test_window:, :]

train_x, train_I, train_R, train_cI, train_cR, train_yI, train_yR, _ = prepare_data(train_feat, active_cases[:, :-valid_window-test_window], recovered_cases[:, :-valid_window-test_window], history_window, pred_window, slide_step)
val_x, val_I, val_R, val_cI, val_cR, val_yI, val_yR, _ = prepare_data(val_feat, active_cases[:, -valid_window-test_window:-test_window], recovered_cases[:, -valid_window-test_window:-test_window], history_window, pred_window, slide_step)
test_x, test_I, test_R, test_cI, test_cR, test_yI, test_yR, test_active_cases = prepare_data(test_feat, active_cases[:, -test_window:], recovered_cases[:, -test_window:], history_window, pred_window, slide_step)

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_x = torch.tensor(train_x).to(device)
train_I = torch.tensor(train_I).to(device)
train_R = torch.tensor(train_R).to(device)
train_cI = torch.tensor(train_cI).to(device)
train_cR = torch.tensor(train_cR).to(device)
train_yI = torch.tensor(train_yI).to(device)
train_yR = torch.tensor(train_yR).to(device)

val_x = torch.tensor(val_x).to(device)
val_I = torch.tensor(val_I).to(device)
val_R = torch.tensor(val_R).to(device)
val_cI = torch.tensor(val_cI).to(device)
val_cR = torch.tensor(val_cR).to(device)
val_yI = torch.tensor(val_yI).to(device)
val_yR = torch.tensor(val_yR).to(device)

test_x = torch.tensor(test_x).to(device)
test_I = torch.tensor(test_I).to(device)
test_R = torch.tensor(test_R).to(device)
test_cI = torch.tensor(test_cI).to(device)
test_cR = torch.tensor(test_cR).to(device)
test_yI = torch.tensor(test_yI).to(device)
test_yR = torch.tensor(test_yR).to(device)
test_active_cases = torch.tensor(test_active_cases).to(device)

dI_mean = torch.tensor(dI_mean, dtype=torch.float32).to(device).reshape((dI_mean.shape[0], 1, 1))
dI_std = torch.tensor(dI_std, dtype=torch.float32).to(device).reshape((dI_mean.shape[0], 1, 1))
dR_mean = torch.tensor(dR_mean, dtype=torch.float32).to(device).reshape((dI_mean.shape[0], 1, 1))
dR_std = torch.tensor(dR_std, dtype=torch.float32).to(device).reshape((dI_mean.shape[0], 1, 1))

N = torch.tensor(static_feat[:, 0], dtype=torch.float32).to(device).unsqueeze(-1)

In [14]:
#Train STAN

# Hyperparameters
in_dim = 3*history_window
hidden_dim1 = 32
hidden_dim2 = 32
gru_dim = 32
num_heads = 1


# Build STAN model for each state
for loc_name in loc_list:
    print("TRAINING NEW LOCATION:", loc_name)
    g = g.to(device)
    model = STAN(g, in_dim, hidden_dim1, hidden_dim2, gru_dim, num_heads, pred_window, device).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    criterion = nn.MSELoss()

    all_loss = []
    file_name = './save/new_data/50_epochs/' + loc_name
    min_loss = 1e10

    cur_loc = loc_list.index(loc_name)

    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        
        active_pred, recovered_pred, phy_active, phy_recover, _ = model(train_x, train_cI[cur_loc], train_cR[cur_loc], N[cur_loc], train_I[cur_loc], train_R[cur_loc])
        phy_active = (phy_active - dI_mean[cur_loc]) / dI_std[cur_loc]
        phy_recover = (phy_recover - dR_mean[cur_loc]) / dR_std[cur_loc]
        loss = criterion(active_pred.squeeze(), train_yI[cur_loc])+criterion(recovered_pred.squeeze(), train_yR[cur_loc])+0.1*criterion(phy_active.squeeze(), train_yI[cur_loc])+0.1*criterion(phy_recover.squeeze(), train_yR[cur_loc])
        
        loss.backward()
        optimizer.step()
        all_loss.append(loss.item())
        
        model.eval()
        _, _, _, _, prev_h = model(train_x, train_cI[cur_loc], train_cR[cur_loc], N[cur_loc], train_I[cur_loc], train_R[cur_loc])
        val_active_pred, val_recovered_pred, val_phy_active, val_phy_recover, _ = model(val_x, val_cI[cur_loc], val_cR[cur_loc], N[cur_loc], val_I[cur_loc], val_R[cur_loc], prev_h)
        
        val_phy_active = (val_phy_active - dI_mean[cur_loc]) / dI_std[cur_loc]
        val_loss = criterion(val_active_pred.squeeze(), val_yI[cur_loc]) + 0.1*criterion(val_phy_active.squeeze(), val_yI[cur_loc])
        if val_loss < min_loss:    
            state = {
                'state': model.state_dict(),
                'optimizer': optimizer.state_dict(),
            }
            torch.save(state, file_name)
            min_loss = val_loss
            print('Saved model')
        
        print('Epoch %d, Loss %.2f, Val loss %.2f'%(epoch, all_loss[-1], val_loss.item()))

TRAINING NEW LOCATION: Alabama


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([1, 15])) that is different to the input size (torch.Size([15])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Saved model
Epoch 0, Loss 3.46, Val loss 0.07
Epoch 1, Loss 5.17, Val loss 0.08
Epoch 2, Loss 6.82, Val loss 0.10
Saved model
Epoch 3, Loss 2.92, Val loss 0.07
Saved model
Epoch 4, Loss 2.46, Val loss 0.05
Saved model
Epoch 5, Loss 2.39, Val loss 0.05
Epoch 6, Loss 2.35, Val loss 0.06
Epoch 7, Loss 2.32, Val loss 0.08
Epoch 8, Loss 2.31, Val loss 0.09
Epoch 9, Loss 2.30, Val loss 0.09
Epoch 10, Loss 2.29, Val loss 0.08
Epoch 11, Loss 2.27, Val loss 0.07
Epoch 12, Loss 2.25, Val loss 0.05
Saved model
Epoch 13, Loss 2.24, Val loss 0.05
Saved model
Epoch 14, Loss 2.23, Val loss 0.04
Epoch 15, Loss 2.21, Val loss 0.04
Epoch 16, Loss 2.20, Val loss 0.05
Epoch 17, Loss 2.20, Val loss 0.06
Epoch 18, Loss 2.19, Val loss 0.07
Epoch 19, Loss 2.18, Val loss 0.07
Epoch 20, Loss 2.17, Val loss 0.07
Epoch 21, Loss 2.16, Val loss 0.06
Epoch 22, Loss 2.15, Val loss 0.06
Epoch 23, Loss 2.14, Val loss 0.06
Epoch 24, Loss 2.14, Val loss 0.06
Epoch 25, Loss 2.13, Val loss 0.06
Epoch 26, Loss 2.12, Val los

In [15]:
#Pred with STAN

model_predictions_number_infected_dictionary = {}
model_predictions_number_infected_list = []

for loc_name in loc_list:
  file_name = './save/new_data/50_epochs/' + loc_name
  checkpoint = torch.load(file_name)
  model = STAN(g, in_dim, hidden_dim1, hidden_dim2, gru_dim, num_heads, pred_window, device).to(device)
  model.load_state_dict(checkpoint['state'])
  model.eval()

  prev_x = torch.cat((train_x, val_x), dim=1)
  prev_I = torch.cat((train_I, val_I), dim=1)
  prev_R = torch.cat((train_R, val_R), dim=1)
  prev_cI = torch.cat((train_cI, val_cI), dim=1)
  prev_cR = torch.cat((train_cR, val_cR), dim=1)

  cur_loc = loc_list.index(loc_name)

  prev_active_pred, _, prev_phyactive_pred, _, h = model(prev_x, prev_cI[cur_loc], prev_cR[cur_loc], N[cur_loc], prev_I[cur_loc], prev_R[cur_loc])

  test_pred_active, test_pred_recovered, test_pred_phy_active, test_pred_phy_recover, _ = model(test_x, test_cI[cur_loc], test_cR[cur_loc], N[cur_loc], test_I[cur_loc], test_R[cur_loc], h)

  # Cumulate predicted dI
  pred_I = []

  for i in range(test_pred_active.size(1)):
      cur_pred = (test_pred_active[0, i, :].detach().cpu().numpy() * dI_std[cur_loc].reshape(1, 1).detach().cpu().numpy()) + dI_mean[cur_loc].reshape(1, 1).detach().cpu().numpy()
      #cur_pred = test_pred_phy_active[0, i, :].detach().cpu().numpy()
      cur_pred = (cur_pred + test_pred_phy_active[0, i, :].detach().cpu().numpy()) / 2
      cur_pred = np.cumsum(cur_pred)
      cur_pred = cur_pred + test_I[cur_loc, i].detach().cpu().item()
      pred_I.append(cur_pred)

  model_predictions_number_infected_list.append(pred_I)  
  # pred_I = np.array(pred_I)
  # pred_I = pred_I

  model_predictions_number_infected_dictionary[loc_name] = np.array(pred_I)[-1, :]

model_predictions_number_infected_tensor = torch.tensor(model_predictions_number_infected_list)
model_predictions_number_infected = model_predictions_number_infected_tensor.reshape((len(loc_list), pred_window)) # (52, 1, 15) --> (52, 15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)


In [ ]:
# Get the ground truth values for number of infected people 
ground_truth_number_infected_dictionary = {}
ground_truth_number_infected_tensor = test_active_cases

test_active_cases_tensor = test_active_cases.reshape((len(loc_list), pred_window)) # (52, 1, 15) --> (52, 15)

for loc_name in loc_list:
  cur_loc = loc_list.index(loc_name)
  ground_truth_number_infected_dictionary[loc_name] = test_active_cases_tensor[cur_loc]


In [17]:
archived_output = {
    'loc_list':loc_list, 
    'model_predictions_number_infected_dictionary':model_predictions_number_infected_dictionary, 
    'model_predictions_number_infected_tensor':model_predictions_number_infected_tensor, # (52, 15)
    'ground_truth_number_infected_dictionary':ground_truth_number_infected_dictionary, 
    'ground_truth_number_infected_tensor':test_active_cases_tensor # (52, 15)
}

import pickle

# Save archived_output as pickle for use later
with open('./save/new_data/50_epochs/archived_output.pickle', 'wb') as handle:
    pickle.dump(archived_output, handle)


# # Load in archived_output pickle
# archived_output = None
# with open('./save/new_data/50_epochs/archived_output.pickle', 'rb') as handle:
#     archived_output = pickle.load(handle)


# print(model_predictions_number_infected['Alabama'])
# print(ground_truth_number_infected['Alabama'])

In [18]:
time_end = time.time()
time_seconds = time_end - time_start
(t_min, t_sec) = divmod(time_seconds,60)
(t_hour,t_min) = divmod(t_min,60) 
print('Time passed: {} hours:{} minutes:{} seconds'.format(t_hour,t_min,t_sec))
# Time passed: 1.0 hours:41.0 minutes:43.83386754989624 seconds

Time passed: 1.0 hours:41.0 minutes:43.83386754989624 seconds
Time passed: 1.0 hours:41.0 minutes:43.83386754989624 seconds
